EoMT
=====


**Your ViT is Secretly an Image Segmentation Model**
 * Paper: https://arxiv.org/abs/2503.19108

![EOMT Comparison](../assets/eomt_comparison.png)

![EOMT Architecture](../assets/eomt_arch.png)